# **Spotify project**
<font size="3">

Introducción al Datascience - Universidad Católica del Norte - 2023

**Estudiantes:**
- **Carlo Ramírez**
- **Joaquín Pinto**
- **Marcelo Céspedes**

Este proyecto tiene como finalidad hacer un **análisis exploratorio** de los datos recopilados sobre las canciones más escuchadas en Spotify 2023 **'spotify-2023.csv'**.
Este conjunto de datos incluye un listado completo de las canciones más famosas del 2023 en Spotify, ofreciendo una amplia gama de características que van más allá de lo habitual en conjuntos similares.

El resumen de los datos que contiene es el siguiente:

</font>

<font size="2">

- **track_name**: Nombre de la canción
- **artist(s)_name**: Nombre del(los) artista(s) de la canción
- **artist_count**: Número de artistas que contribuyen a la canción
- **released_year**: Año de lanzamiento de la canción
- **released_month**: Mes de lanzamiento de la canción
- **released_day**: Día del mes en que se lanzó la canción
- **in_spotify_playlists**: Número de listas de reproducción de Spotify en las que se incluye la canción
- **in_spotify_charts**: Presencia y posición de la canción en las listas de Spotify
- **streams**: Número total de reproducciones en Spotify
- **in_apple_playlists**: Número de listas de reproducción de Apple Music en las que se incluye la canción
- **in_apple_charts**: Presencia y posición de la canción en las listas de Apple Music
- **in_deezer_playlists**: Número de listas de reproducción de Deezer en las que se incluye la canción
- **in_deezer_charts**: Presencia y posición de la canción en las listas de Deezer
- **in_shazam_charts**: Presencia y posición de la canción en las listas de Shazam
- **bpm**: Latidos por minuto, una medida del tempo de la canción
- **key**: Tonalidad de la canción
- **mode**: Modo de la canción (mayor o menor)
- **danceability%**: Porcentaje que indica la idoneidad de la canción para bailar
- **valence_%**: Positividad del contenido musical de la canción
- **energy_%**: Nivel percibido de energía de la canción
- **acousticness_%**: Cantidad de sonido acústico en la canción
- **instrumentalness_%**: Cantidad de contenido instrumental en la canción
- **liveness_%**: Presencia de elementos de actuación en vivo
- **speechiness_%**: Cantidad de palabras habladas en la canción

</font>

## #0 Librerías
<font size="2">

Se importan las librerías a utilizar en el análisis.
</font>

In [117]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

## #1 Carga de datos
<font size="2">

Se cargan los datos de **'spotify-2023.csv'** en un dataframe usando pandas.
</font>

In [118]:
# Cargar los datos
file_path = './spotify-2023.csv'
spotify_data = pd.read_csv(file_path, encoding='ISO-8859-1')

## #2 Resumen inicial de los datos
<font size="2">

Se describen los datos cargados para tener una visualización inicial y poder trabajarlos.
</font>

In [119]:
# Definimos las columnas que son números
number_cols = ['artist_count', 'released_year', 'released_month', 'released_day', 'bpm', 'streams',
               'in_spotify_playlists', 'in_apple_playlists', 'in_deezer_playlists', 'in_spotify_charts', 'in_apple_charts', 'in_deezer_charts', 'in_shazam_charts',  
               'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']

# Limpiamos valores faltantes previamente visualizados
spotify_data = spotify_data.dropna(subset=['key'])

# Luego convertimos cada columna en number_cols a entero y rellenamos con 0 en caso de ser necesario
for col in number_cols:
    spotify_data[col] = pd.to_numeric(spotify_data[col], downcast='integer', errors='coerce')
    spotify_data[col] = spotify_data[col].fillna(0)
    spotify_data[col] = spotify_data[col].astype('int64')

<font size="2">

Se puede observar que aproximadamente desde el **2021** al **2023** hay grandes cantidades de datos, pero para atrás no muchos (o no tan significativos), por lo tanto el análisis tendrá más influencia por los datos en este rango de tiempo.
</font>

<font size="2">

Se puede visualizar aquí una relación proporcional entre el % de **energía**, y el % de **danzabilidad** que tienen las canciones, factor que tiene sentido en el conjunto de datos.
</font>

## #5 Normalización de datos

<font size="2">

Para analizar los datos de forma más coherente en magnitud, se normalizarán las columnas numéricas relevantes identificadas.
</font>

In [120]:
# Normalizar todas las columnas relevantes
scaler = MinMaxScaler()
to_normalize_cols = ['in_spotify_playlists', 'in_spotify_charts', 'streams',
                         'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists',
                         'in_deezer_charts', 'in_shazam_charts', 'bpm']

percentage_columns = ['danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']

# Escalar los valores de porcentaje a una escala de 0 a 1
spotify_data[percentage_columns] = spotify_data[percentage_columns] / 100.0

spotify_data_normalized = spotify_data.copy()
spotify_data_normalized[to_normalize_cols] = scaler.fit_transform(spotify_data[to_normalize_cols])

## #6 Análisis de correlación entre la cantidad de **streams** y la presencia en plataformas digitales

<font size="2">

Se visualizará cuál es la correlación que existe entre la cantidad de streams y la presencia en diversas plataformas digitales, en este caso **Spotify**, **Apple**, **Deezer** y **Shazam**.
</font>

<font size="2">

Se puede observar que la presencia de cada canción en las playlist de **Spotify**, **Apple** y **Deezer** (aunque menor) parece tener un gran impacto en la cantidad de streams que alcanza cada canción.
</font>

In [131]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor

# Invertir las características con relación inversa a streams
spotify_data_normalized['inv_acousticness'] = 1 - spotify_data_normalized['acousticness_%']
spotify_data_normalized['inv_instrumentalness'] = 1 - spotify_data_normalized['instrumentalness_%']
spotify_data_normalized['inv_liveness'] = 1 - spotify_data_normalized['liveness_%']
spotify_data_normalized['inv_speechiness'] = 1 - spotify_data_normalized['speechiness_%']

# Seleccionar las características (variables independientes) para el modelo
features = spotify_data_normalized[['in_spotify_playlists', 'in_spotify_charts', 'streams',
                         'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists',
                         'in_deezer_charts', 'in_shazam_charts', 'bpm', 'danceability_%',
                         'valence_%', 'energy_%', 'inv_acousticness', 'inv_instrumentalness',
                         'inv_liveness', 'inv_speechiness']]

# Seleccionar las variables dependientes (target)
target_columns = ['streams']
target = spotify_data_normalized[target_columns]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.4, random_state=42)

sm1 = XGBRegressor(random_state=42, n_estimators=200, learning_rate=0.3, reg_lambda=0.5, reg_alpha=0.25)
%time sm1.fit(X_train, y_train)
y_pred = sm1.predict(X_test)
r2_1 = r2_score(y_test, y_pred)
mae_1 = mean_absolute_error(y_test, y_pred)
rmse_1 = mean_squared_error(y_test, y_pred, squared=False)
print(f'SM1: R2 = {r2_1}, MAE = {mae_1}, RMSE = {rmse_1}')

CPU times: total: 203 ms
Wall time: 67.7 ms
SM1: R2 = 0.9881325302924402, MAE = 0.005479185151101074, RMSE = 0.017510116882462384


In [130]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor

# Invertir las características con relación inversa a streams
spotify_data_normalized['inv_acousticness'] = 1 - spotify_data_normalized['acousticness_%']
spotify_data_normalized['inv_instrumentalness'] = 1 - spotify_data_normalized['instrumentalness_%']
spotify_data_normalized['inv_liveness'] = 1 - spotify_data_normalized['liveness_%']
spotify_data_normalized['inv_speechiness'] = 1 - spotify_data_normalized['speechiness_%']

# Seleccionar las características (variables independientes) para el modelo
features = spotify_data_normalized[['in_spotify_playlists', 'in_spotify_charts', 'streams',
                         'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists',
                         'in_deezer_charts', 'in_shazam_charts', 'bpm', 'danceability_%',
                         'valence_%', 'energy_%', 'inv_acousticness', 'inv_instrumentalness',
                         'inv_liveness', 'inv_speechiness']]

# Seleccionar las variables dependientes (target)
target_columns = ['streams']
target = spotify_data_normalized[target_columns]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.4, random_state=42)

sm1 = RandomForestRegressor(n_estimators=100, random_state=42)
%time sm1.fit(X_train, y_train)
y_pred = sm1.predict(X_test)
r2_1 = r2_score(y_test, y_pred)
mae_1 = mean_absolute_error(y_test, y_pred)
rmse_1 = mean_squared_error(y_test, y_pred, squared=False)
print(f'SM1: R2 = {r2_1}, MAE = {mae_1}, RMSE = {rmse_1}')

C:\Users\mcesp\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


CPU times: total: 188 ms
Wall time: 1.13 s
SM1: R2 = 0.9945277577046155, MAE = 0.002051517825002513, RMSE = 0.011890295238870021
